<a href="https://colab.research.google.com/github/RohanGforwork/ML_text_summarization/blob/main/Translation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

!pip install googletrans==4.0.0-rc1 better_profanity nltk



Language translation


In [5]:
import nltk
from googletrans import Translator
from better_profanity import profanity
from nltk.corpus import wordnet

nltk.download('wordnet')

class LanguageTranslator:
    def __init__(self):
        self.translator = Translator()

    def is_slang(self, text):
        """
        Checks if the given text contains slang or informal words.
        :param text: The input text.
        :return: Boolean (True if slang is detected, False otherwise)
        """
        words = text.split()
        for word in words:
            if not wordnet.synsets(word):
                return True  # If no dictionary meaning, assume slang
            if profanity.contains_profanity(word):
                return True  # If flagged as profane/slang, return True
        return False

    def detect_and_translate(self, text):
        """
        Detects the language, translates to English if needed, and checks for slang.
        :param text: User input text.
        :return: Dictionary with detected language, confidence, translation, and slang detection.
        """
        detected = self.translator.detect(text)

        # Handle NoneType confidence score
        confidence_score = round(detected.confidence, 2) if detected.confidence else "Unknown"

        response = {
            "detected_language": detected.lang,
            "confidence": confidence_score,
            "original_text": text,
            "translated_text": text,
            "contains_slang": self.is_slang(text)
        }

        if detected.lang != 'en':
            translated = self.translator.translate(text, dest='en')
            response["translated_text"] = translated.text

        return response

# Example usage (for testing only)
if __name__ == "__main__":
    translator = LanguageTranslator()
    sample_text = input("Enter text: ")
    result = translator.detect_and_translate(sample_text)

    print(f"Detected Language: {result['detected_language']} (Confidence: {result['confidence']})")
    print(f"Original Text: {result['original_text']}")
    print(f"Translated Text: {result['translated_text']}")
    print(f"Contains Slang: {result['contains_slang']}")


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Enter text: hegiddira!
Detected Language: kn (Confidence: Unknown)
Original Text: hegiddira!
Translated Text: How are you!
Contains Slang: True
